<a href="https://colab.research.google.com/github/jqzba/Cryptocurrency-analysis/blob/main/Cryptopredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

!pip install yfinance
from google.colab import files

import yfinance as yf
from datetime import datetime
from datetime import timedelta

#load the data

today = datetime.today().strftime('%Y-%m-%d')
start_date = '2018-01-01'
eth_df = yf.download('ETH-USD',start_date, today)
eth_df.tail()
eth_df = eth_df.reset_index()

print(eth_df.iloc[:, 0])

     |████████████████████████████████| 6.4 MB 7.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
[*********************100%***********************]  1 of 1 completed
0      2018-01-01
1      2018-01-02
2      2018-01-03
3      2018-01-04
4      2018-01-05
          ...    
1586   2022-05-06
1587   2022-05-07
15

In [ ]:
main_df = pd.DataFrame()


ratios = ["BTC-USD", "ETH-USD", "LINK-USD"]

for i in ratios:
  df = yf.download(i,start_date, today)
  df = df.reset_index()
  df.rename(columns={"Close": f"{i}_close", "Volume":f"{i}_volume"}, inplace=True)
  
  
  print(list(df.columns))
  df = df[['Date', f"{i}_close", f"{i}_volume"]]
  

  if len(main_df) == 0:
    main_df = df
    print("jee")
  else:
    main_df.Date.astype(int)
    main_df = main_df.merge(df, on='Date', how='left')
    print(df)

print(main_df.tail())



[*********************100%***********************]  1 of 1 completed
['Date', 'Open', 'High', 'Low', 'BTC-USD_close', 'Adj Close', 'BTC-USD_volume']
jee
[*********************100%***********************]  1 of 1 completed
['Date', 'Open', 'High', 'Low', 'ETH-USD_close', 'Adj Close', 'ETH-USD_volume']
           Date  ETH-USD_close  ETH-USD_volume
0    2018-01-01     772.640991      2595760128
1    2018-01-02     884.443970      5783349760
2    2018-01-03     962.719971      5093159936
3    2018-01-04     980.921997      6502859776
4    2018-01-05     997.719971      6683149824
...         ...            ...             ...
1586 2022-05-06    2694.979736     21027599270
1587 2022-05-07    2636.093018     13369276367
1588 2022-05-08    2517.459961     20802269935
1589 2022-05-09    2245.430420     36333567678
1590 2022-05-11    2356.415283     37408894976

[1591 rows x 3 columns]
[*********************100%***********************]  1 of 1 completed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



['Date', 'Open', 'High', 'Low', 'LINK-USD_close', 'Adj Close', 'LINK-USD_volume']
           Date  LINK-USD_close  LINK-USD_volume
0    2018-01-01        0.733563         11506300
1    2018-01-02        0.673712         12671100
2    2018-01-03        0.681167          8620400
3    2018-01-04        0.984368         24679000
4    2018-01-05        0.907486         23599300
...         ...             ...              ...
1586 2022-05-06       10.799844        507557055
1587 2022-05-07       10.239905        354478668
1588 2022-05-08        9.990030        427837348
1589 2022-05-09        8.272897        844443662
1590 2022-05-11        8.666528        972119616

[1591 rows x 3 columns]
           Date  BTC-USD_close  BTC-USD_volume  ETH-USD_close  ETH-USD_volume  \
1586 2022-05-06   36040.921875     37795577489    2694.979736     21027599270   
1587 2022-05-07   35501.953125     24375896406    2636.093018     13369276367   
1588 2022-05-08   34059.265625     36763041910    2517.459961

In [ ]:
import os
from sklearn import preprocessing
from collections import deque

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 5
RATIO_TO_PREDICT = "LINK-USD"

def classify(current, future):
  if float(future) > float(current):
    return 1
  else:
    return 0

def preprocess_df(df):
  df = df.drop('future', 1)

  for col in df.columns: 
    if col != "target":

      #df.set_index('Date')
      #df[col] = df[col].pct_change().apply(lambda x: np.log(x+1)).cov
      df.drop(['Date'], axis=1)
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values)

  df.dropna(inplace=True)

  sequential_data = []
  prev_days = deque(maxlen=SEQ_LEN)
  print(df.head())

  for c in df.columns:
    print(c) 



main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

#print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))

times=sorted(main_df.index.values)

last_5pct = times[-int(0.05*len(times))]

print(last_5pct)

validation_main_df = main_df[(main_df.index >= last_5pct)]
validation_main_df = main_df[(main_df.index < last_5pct)]


train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(main_df)




1512
       Date  BTC-USD_close  BTC-USD_volume  ETH-USD_close  ETH-USD_volume  \
0 -1.730960      -0.361938       -0.764001      -0.258273       -0.929006   
1 -1.728782      -0.288830       -0.443853      -0.170583       -0.642948   
2 -1.726605      -0.276751       -0.442618      -0.109189       -0.704886   
3 -1.724428      -0.254778       -0.202764      -0.094912       -0.578378   
4 -1.722251      -0.153781       -0.102272      -0.081737       -0.562199   

   LINK-USD_close  LINK-USD_volume  target  
0       -0.798134        -0.182473       1  
1       -0.803525        -0.182206       1  
2       -0.802853        -0.183135       1  
3       -0.775543        -0.179453       1  
4       -0.782468        -0.179701       1  
Date
BTC-USD_close
BTC-USD_volume
ETH-USD_close
ETH-USD_volume
LINK-USD_close
LINK-USD_volume
target


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large

TypeError: ignored